In [1]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 1

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


0 1
1 0
 1 
 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 1
place: 2 	frac:	 1 / 0

Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  3.236817000015435e-06  seconds.
At a depth of  1  layers.


In [2]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 2

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


0   1
1   0
  1  
  1  
 1 2 
 2 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 2
place: 2 	frac:	 1 / 1
place: 3 	frac:	 2 / 1
place: 4 	frac:	 1 / 0

Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  5.667478899977141e-06  seconds.
At a depth of  2  layers.


In [3]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 3

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


0       1
1       0
    1    
    1    
  1   2  
  2   1  
 1 2 3 3 
 3 3 2 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 3
place: 2 	frac:	 1 / 2
place: 3 	frac:	 2 / 3
place: 4 	frac:	 1 / 1
place: 5 	frac:	 3 / 2
place: 6 	frac:	 2 / 1
place: 7 	frac:	 3 / 1
place: 8 	frac:	 1 / 0

Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  1.0675343699949735e-05  seconds.
At a depth of  3  layers.


In [4]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 4

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


0               1
1               0
        1        
        1        
    1       2    
    2       1    
  1   2   3   3  
  3   3   2   1  
 1 2 3 3 4 5 5 4 
 4 5 5 4 3 3 2 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 4
place: 2 	frac:	 1 / 3
place: 3 	frac:	 2 / 5
place: 4 	frac:	 1 / 2
place: 5 	frac:	 3 / 5
place: 6 	frac:	 2 / 3
place: 7 	frac:	 3 / 4
place: 8 	frac:	 1 / 1
place: 9 	frac:	 4 / 3
place: 10 	frac:	 3 / 2
place: 11 	frac:	 5 / 3
place: 12 	frac:	 2 / 1
place: 13 	frac:	 5 / 2
place: 14 	frac:	 3 / 1
place: 15 	frac:	 4 / 1
place: 16 	frac:	 1 / 0

Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  2.2199758300030226e-05  seconds.
At a depth of  4  layers.


In [5]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 4

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


0               1
1               0
        1        
        1        
    1       2    
    2       1    
  1   2   3   3  
  3   3   2   1  
 1 2 3 3 4 5 5 4 
 4 5 5 4 3 3 2 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 4
place: 2 	frac:	 1 / 3
place: 3 	frac:	 2 / 5
place: 4 	frac:	 1 / 2
place: 5 	frac:	 3 / 5
place: 6 	frac:	 2 / 3
place: 7 	frac:	 3 / 4
place: 8 	frac:	 1 / 1
place: 9 	frac:	 4 / 3
place: 10 	frac:	 3 / 2
place: 11 	frac:	 5 / 3
place: 12 	frac:	 2 / 1
place: 13 	frac:	 5 / 2
place: 14 	frac:	 3 / 1
place: 15 	frac:	 4 / 1
place: 16 	frac:	 1 / 0

Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  1.6392886800076668e-05  seconds.
At a depth of  4  layers.


In [6]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 5

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


0                               1
1                               0
                1                
                1                
        1               2        
        2               1        
    1       2       3       3    
    3       3       2       1    
  1   2   3   3   4   5   5   4  
  4   5   5   4   3   3   2   1  
 1 2 3 3 4 5 5 4 5 7 8 7 7 8 7 5 
 5 7 8 7 7 8 7 5 4 5 5 4 3 3 2 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 5
place: 2 	frac:	 1 / 4
place: 3 	frac:	 2 / 7
place: 4 	frac:	 1 / 3
place: 5 	frac:	 3 / 8
place: 6 	frac:	 2 / 5
place: 7 	frac:	 3 / 7
place: 8 	frac:	 1 / 2
place: 9 	frac:	 4 / 7
place: 10 	frac:	 3 / 5
place: 11 	frac:	 5 / 8
place: 12 	frac:	 2 / 3
place: 13 	frac:	 5 / 7
place: 14 	frac:	 3 / 4
place: 15 	frac:	 4 / 5
place: 16 	frac:	 1 / 1
place: 17 	frac:	 5 / 4
place: 18 	frac:	 4 / 3
place: 19 	frac:	 7 / 5
place: 20 	frac:	 3 / 2
place: 21 	frac:	 8 / 5
place: 22 	frac:	 5 / 3
place: 23 	frac:	 7 / 4
place: 24 	frac:	 2 / 1


In [7]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 10

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    print("No tree - too big")
    #draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


No tree - too big


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 10
place: 2 	frac:	 1 / 9
place: 3 	frac:	 2 / 17
place: 4 	frac:	 1 / 8
place: 5 	frac:	 3 / 23
place: 6 	frac:	 2 / 15
place: 7 	frac:	 3 / 22
place: 8 	frac:	 1 / 7
place: 9 	frac:	 4 / 27
place: 10 	frac:	 3 / 20
place: 11 	frac:	 5 / 33
place: 12 	frac:	 2 / 13
place: 13 	frac:	 5 / 32
place: 14 	frac:	 3 / 19
place: 15 	frac:	 4 / 25
place: 16 	frac:	 1 / 6
place: 17 	frac:	 5 / 29
place: 18 	frac:	 4 / 23
place: 19 	frac:	 7 / 40
place: 20 	frac:	 3 / 17
place: 21 	frac:	 8 / 45
place: 22 	frac:	 5 / 28
place: 23 	frac:	 7 / 39
place: 24 	frac:	 2 / 11
place: 25 	frac:	 7 / 38
place: 26 	frac:	 5 / 27
place: 27 	frac:	 8 / 43
place: 28 	frac:	 3 / 16
place: 29 	frac:	 7 / 37
place: 30 	frac:	 4 / 21
place: 31 	frac:	 5 / 26
place: 32 	frac:	 1 / 5
place: 33 	frac:	 6 / 29
place: 34 	frac:	 5 / 24
place: 35 	frac:	 9 / 43
place: 36 	frac:	 4 / 19
place: 37 	frac:	 11 / 52
place: 38 	frac:	 7 / 33
place: 39 	frac:	 10

place: 617 	frac:	 61 / 43
place: 618 	frac:	 44 / 31
place: 619 	frac:	 71 / 50
place: 620 	frac:	 27 / 19
place: 621 	frac:	 64 / 45
place: 622 	frac:	 37 / 26
place: 623 	frac:	 47 / 33
place: 624 	frac:	 10 / 7
place: 625 	frac:	 43 / 30
place: 626 	frac:	 33 / 23
place: 627 	frac:	 56 / 39
place: 628 	frac:	 23 / 16
place: 629 	frac:	 59 / 41
place: 630 	frac:	 36 / 25
place: 631 	frac:	 49 / 34
place: 632 	frac:	 13 / 9
place: 633 	frac:	 42 / 29
place: 634 	frac:	 29 / 20
place: 635 	frac:	 45 / 31
place: 636 	frac:	 16 / 11
place: 637 	frac:	 35 / 24
place: 638 	frac:	 19 / 13
place: 639 	frac:	 22 / 15
place: 640 	frac:	 3 / 2
place: 641 	frac:	 23 / 15
place: 642 	frac:	 20 / 13
place: 643 	frac:	 37 / 24
place: 644 	frac:	 17 / 11
place: 645 	frac:	 48 / 31
place: 646 	frac:	 31 / 20
place: 647 	frac:	 45 / 29
place: 648 	frac:	 14 / 9
place: 649 	frac:	 53 / 34
place: 650 	frac:	 39 / 25
place: 651 	frac:	 64 / 41
place: 652 	frac:	 25 / 16
place: 653 	frac:	 61 / 39
place:


Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  0.0010363221415000225  seconds.
At a depth of  10  layers.


In [8]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 18 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 12

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    print("No tree - too big")
    #draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of ", depth, " layers.")


No tree - too big


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 12
place: 2 	frac:	 1 / 11
place: 3 	frac:	 2 / 21
place: 4 	frac:	 1 / 10
place: 5 	frac:	 3 / 29
place: 6 	frac:	 2 / 19
place: 7 	frac:	 3 / 28
place: 8 	frac:	 1 / 9
place: 9 	frac:	 4 / 35
place: 10 	frac:	 3 / 26
place: 11 	frac:	 5 / 43
place: 12 	frac:	 2 / 17
place: 13 	frac:	 5 / 42
place: 14 	frac:	 3 / 25
place: 15 	frac:	 4 / 33
place: 16 	frac:	 1 / 8
place: 17 	frac:	 5 / 39
place: 18 	frac:	 4 / 31
place: 19 	frac:	 7 / 54
place: 20 	frac:	 3 / 23
place: 21 	frac:	 8 / 61
place: 22 	frac:	 5 / 38
place: 23 	frac:	 7 / 53
place: 24 	frac:	 2 / 15
place: 25 	frac:	 7 / 52
place: 26 	frac:	 5 / 37
place: 27 	frac:	 8 / 59
place: 28 	frac:	 3 / 22
place: 29 	frac:	 7 / 51
place: 30 	frac:	 4 / 29
place: 31 	frac:	 5 / 36
place: 32 	frac:	 1 / 7
place: 33 	frac:	 6 / 41
place: 34 	frac:	 5 / 34
place: 35 	frac:	 9 / 61
place: 36 	frac:	 4 / 27
place: 37 	frac:	 11 / 74
place: 38 	frac:	 7 / 47
place: 39 	frac:	 

place: 447 	frac:	 20 / 67
place: 448 	frac:	 3 / 10
place: 449 	frac:	 19 / 63
place: 450 	frac:	 16 / 53
place: 451 	frac:	 29 / 96
place: 452 	frac:	 13 / 43
place: 453 	frac:	 36 / 119
place: 454 	frac:	 23 / 76
place: 455 	frac:	 33 / 109
place: 456 	frac:	 10 / 33
place: 457 	frac:	 37 / 122
place: 458 	frac:	 27 / 89
place: 459 	frac:	 44 / 145
place: 460 	frac:	 17 / 56
place: 461 	frac:	 41 / 135
place: 462 	frac:	 24 / 79
place: 463 	frac:	 31 / 102
place: 464 	frac:	 7 / 23
place: 465 	frac:	 32 / 105
place: 466 	frac:	 25 / 82
place: 467 	frac:	 43 / 141
place: 468 	frac:	 18 / 59
place: 469 	frac:	 47 / 154
place: 470 	frac:	 29 / 95
place: 471 	frac:	 40 / 131
place: 472 	frac:	 11 / 36
place: 473 	frac:	 37 / 121
place: 474 	frac:	 26 / 85
place: 475 	frac:	 41 / 134
place: 476 	frac:	 15 / 49
place: 477 	frac:	 34 / 111
place: 478 	frac:	 19 / 62
place: 479 	frac:	 23 / 75
place: 480 	frac:	 4 / 13
place: 481 	frac:	 21 / 68
place: 482 	frac:	 17 / 55
place: 483 	frac:	

place: 997 	frac:	 43 / 94
place: 998 	frac:	 27 / 59
place: 999 	frac:	 38 / 83
place: 1000 	frac:	 11 / 24
place: 1001 	frac:	 39 / 85
place: 1002 	frac:	 28 / 61
place: 1003 	frac:	 45 / 98
place: 1004 	frac:	 17 / 37
place: 1005 	frac:	 40 / 87
place: 1006 	frac:	 23 / 50
place: 1007 	frac:	 29 / 63
place: 1008 	frac:	 6 / 13
place: 1009 	frac:	 25 / 54
place: 1010 	frac:	 19 / 41
place: 1011 	frac:	 32 / 69
place: 1012 	frac:	 13 / 28
place: 1013 	frac:	 33 / 71
place: 1014 	frac:	 20 / 43
place: 1015 	frac:	 27 / 58
place: 1016 	frac:	 7 / 15
place: 1017 	frac:	 22 / 47
place: 1018 	frac:	 15 / 32
place: 1019 	frac:	 23 / 49
place: 1020 	frac:	 8 / 17
place: 1021 	frac:	 17 / 36
place: 1022 	frac:	 9 / 19
place: 1023 	frac:	 10 / 21
place: 1024 	frac:	 1 / 2
place: 1025 	frac:	 11 / 21
place: 1026 	frac:	 10 / 19
place: 1027 	frac:	 19 / 36
place: 1028 	frac:	 9 / 17
place: 1029 	frac:	 26 / 49
place: 1030 	frac:	 17 / 32
place: 1031 	frac:	 25 / 47
place: 1032 	frac:	 8 / 15
pla

place: 1469 	frac:	 73 / 115
place: 1470 	frac:	 40 / 63
place: 1471 	frac:	 47 / 74
place: 1472 	frac:	 7 / 11
place: 1473 	frac:	 44 / 69
place: 1474 	frac:	 37 / 58
place: 1475 	frac:	 67 / 105
place: 1476 	frac:	 30 / 47
place: 1477 	frac:	 83 / 130
place: 1478 	frac:	 53 / 83
place: 1479 	frac:	 76 / 119
place: 1480 	frac:	 23 / 36
place: 1481 	frac:	 85 / 133
place: 1482 	frac:	 62 / 97
place: 1483 	frac:	 101 / 158
place: 1484 	frac:	 39 / 61
place: 1485 	frac:	 94 / 147
place: 1486 	frac:	 55 / 86
place: 1487 	frac:	 71 / 111
place: 1488 	frac:	 16 / 25
place: 1489 	frac:	 73 / 114
place: 1490 	frac:	 57 / 89
place: 1491 	frac:	 98 / 153
place: 1492 	frac:	 41 / 64
place: 1493 	frac:	 107 / 167
place: 1494 	frac:	 66 / 103
place: 1495 	frac:	 91 / 142
place: 1496 	frac:	 25 / 39
place: 1497 	frac:	 84 / 131
place: 1498 	frac:	 59 / 92
place: 1499 	frac:	 93 / 145
place: 1500 	frac:	 34 / 53
place: 1501 	frac:	 77 / 120
place: 1502 	frac:	 43 / 67
place: 1503 	frac:	 52 / 81
pla

place: 2097 	frac:	 58 / 49
place: 2098 	frac:	 45 / 38
place: 2099 	frac:	 77 / 65
place: 2100 	frac:	 32 / 27
place: 2101 	frac:	 83 / 70
place: 2102 	frac:	 51 / 43
place: 2103 	frac:	 70 / 59
place: 2104 	frac:	 19 / 16
place: 2105 	frac:	 63 / 53
place: 2106 	frac:	 44 / 37
place: 2107 	frac:	 69 / 58
place: 2108 	frac:	 25 / 21
place: 2109 	frac:	 56 / 47
place: 2110 	frac:	 31 / 26
place: 2111 	frac:	 37 / 31
place: 2112 	frac:	 6 / 5
place: 2113 	frac:	 41 / 34
place: 2114 	frac:	 35 / 29
place: 2115 	frac:	 64 / 53
place: 2116 	frac:	 29 / 24
place: 2117 	frac:	 81 / 67
place: 2118 	frac:	 52 / 43
place: 2119 	frac:	 75 / 62
place: 2120 	frac:	 23 / 19
place: 2121 	frac:	 86 / 71
place: 2122 	frac:	 63 / 52
place: 2123 	frac:	 103 / 85
place: 2124 	frac:	 40 / 33
place: 2125 	frac:	 97 / 80
place: 2126 	frac:	 57 / 47
place: 2127 	frac:	 74 / 61
place: 2128 	frac:	 17 / 14
place: 2129 	frac:	 79 / 65
place: 2130 	frac:	 62 / 51
place: 2131 	frac:	 107 / 88
place: 2132 	frac:	 

place: 2675 	frac:	 151 / 95
place: 2676 	frac:	 62 / 39
place: 2677 	frac:	 159 / 100
place: 2678 	frac:	 97 / 61
place: 2679 	frac:	 132 / 83
place: 2680 	frac:	 35 / 22
place: 2681 	frac:	 113 / 71
place: 2682 	frac:	 78 / 49
place: 2683 	frac:	 121 / 76
place: 2684 	frac:	 43 / 27
place: 2685 	frac:	 94 / 59
place: 2686 	frac:	 51 / 32
place: 2687 	frac:	 59 / 37
place: 2688 	frac:	 8 / 5
place: 2689 	frac:	 61 / 38
place: 2690 	frac:	 53 / 33
place: 2691 	frac:	 98 / 61
place: 2692 	frac:	 45 / 28
place: 2693 	frac:	 127 / 79
place: 2694 	frac:	 82 / 51
place: 2695 	frac:	 119 / 74
place: 2696 	frac:	 37 / 23
place: 2697 	frac:	 140 / 87
place: 2698 	frac:	 103 / 64
place: 2699 	frac:	 169 / 105
place: 2700 	frac:	 66 / 41
place: 2701 	frac:	 161 / 100
place: 2702 	frac:	 95 / 59
place: 2703 	frac:	 124 / 77
place: 2704 	frac:	 29 / 18
place: 2705 	frac:	 137 / 85
place: 2706 	frac:	 108 / 67
place: 2707 	frac:	 187 / 116
place: 2708 	frac:	 79 / 49
place: 2709 	frac:	 208 / 129
p

place: 3190 	frac:	 83 / 36
place: 3191 	frac:	 113 / 49
place: 3192 	frac:	 30 / 13
place: 3193 	frac:	 97 / 42
place: 3194 	frac:	 67 / 29
place: 3195 	frac:	 104 / 45
place: 3196 	frac:	 37 / 16
place: 3197 	frac:	 81 / 35
place: 3198 	frac:	 44 / 19
place: 3199 	frac:	 51 / 22
place: 3200 	frac:	 7 / 3
place: 3201 	frac:	 54 / 23
place: 3202 	frac:	 47 / 20
place: 3203 	frac:	 87 / 37
place: 3204 	frac:	 40 / 17
place: 3205 	frac:	 113 / 48
place: 3206 	frac:	 73 / 31
place: 3207 	frac:	 106 / 45
place: 3208 	frac:	 33 / 14
place: 3209 	frac:	 125 / 53
place: 3210 	frac:	 92 / 39
place: 3211 	frac:	 151 / 64
place: 3212 	frac:	 59 / 25
place: 3213 	frac:	 144 / 61
place: 3214 	frac:	 85 / 36
place: 3215 	frac:	 111 / 47
place: 3216 	frac:	 26 / 11
place: 3217 	frac:	 123 / 52
place: 3218 	frac:	 97 / 41
place: 3219 	frac:	 168 / 71
place: 3220 	frac:	 71 / 30
place: 3221 	frac:	 187 / 79
place: 3222 	frac:	 116 / 49
place: 3223 	frac:	 161 / 68
place: 3224 	frac:	 45 / 19
place: 32

place: 3773 	frac:	 113 / 31
place: 3774 	frac:	 62 / 17
place: 3775 	frac:	 73 / 20
place: 3776 	frac:	 11 / 3
place: 3777 	frac:	 70 / 19
place: 3778 	frac:	 59 / 16
place: 3779 	frac:	 107 / 29
place: 3780 	frac:	 48 / 13
place: 3781 	frac:	 133 / 36
place: 3782 	frac:	 85 / 23
place: 3783 	frac:	 122 / 33
place: 3784 	frac:	 37 / 10
place: 3785 	frac:	 137 / 37
place: 3786 	frac:	 100 / 27
place: 3787 	frac:	 163 / 44
place: 3788 	frac:	 63 / 17
place: 3789 	frac:	 152 / 41
place: 3790 	frac:	 89 / 24
place: 3791 	frac:	 115 / 31
place: 3792 	frac:	 26 / 7
place: 3793 	frac:	 119 / 32
place: 3794 	frac:	 93 / 25
place: 3795 	frac:	 160 / 43
place: 3796 	frac:	 67 / 18
place: 3797 	frac:	 175 / 47
place: 3798 	frac:	 108 / 29
place: 3799 	frac:	 149 / 40
place: 3800 	frac:	 41 / 11
place: 3801 	frac:	 138 / 37
place: 3802 	frac:	 97 / 26
place: 3803 	frac:	 153 / 41
place: 3804 	frac:	 56 / 15
place: 3805 	frac:	 127 / 34
place: 3806 	frac:	 71 / 19
place: 3807 	frac:	 86 / 23
place


Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  0.004621188801599964  seconds.
At a depth of  12  layers.


In [9]:
def test_count_nodes():
    depth = 0
    node_count = count_nodes(depth)    
    assert (node_count == 0), 'inaccurate node count'
    
    depth = 1
    node_count = count_nodes(depth)
    assert (node_count == 1), 'inaccurate node count'
    
    depth = 2
    node_count = count_nodes(depth)
    assert (node_count == 3), 'inaccurate node count'
    
    depth = 10
    node_count = count_nodes(depth)
    assert (node_count == 1023), 'inaccurate node count'
    
    depth = 21
    node_count = count_nodes(depth)
    assert (node_count == 2097151), 'inaccurate node count'
    
def PythonSternBrocotTests():
    test_count_nodes()
    print("passed all tests")
    
PythonSternBrocotTests()

passed all tests
